In [1]:
import torch

In [2]:
torch.__version__

'1.8.1'

In [3]:
torch.__file__

'/apps/pytorch/1.8.1/lib/python3.9/site-packages/torch/__init__.py'

In [4]:
import sys
sys.path.insert(1, '/home/juannanzhou/.local/lib/python3.9/site-packages')
sys.path.insert(1, '/home/juannanzhou/.local/bin')

import itertools
import torch
import torch.nn.functional as F
import numpy as np
import GPUtil

n_devices = torch.cuda.device_count()
output_device = torch.device('cuda:0')
print('Planning to run on {} GPUs.'.format(n_devices))

GPUtil.showUtilization()

Planning to run on 8 GPUs.
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  0% |  0% |
|  2 |  0% |  0% |
|  3 |  0% |  0% |
|  4 |  0% |  0% |
|  5 |  0% |  0% |
|  6 |  0% |  0% |
|  7 |  0% |  0% |


In [5]:
alpha = 4
l = 8
alphabet = list(range(alpha))

# prob no mutation at time 1
# q = 1 - 1/l
q = 0.7

odds = torch.tensor([q**t/(1 - q**t) for t in range(1, l+1)]).to(output_device)

scaling_factors = torch.tensor([(1 - q**t)**l for t in range(l+1)]).to(output_device)
scaling_factors[0] = 1

seqs = list(itertools.product(alphabet, repeat=l))
seqs = torch.tensor(seqs).to(output_device)
seqs1h_test = torch.flatten(torch.tensor(F.one_hot(seqs), dtype=torch.float32), start_dim=1).to(output_device)

seqs2 = []
for d in range(0, l+1):
    seq = torch.zeros(l)
    seq[:d] = 3
    seqs2.append(seq)

seqs2 = torch.stack(seqs2).type(torch.int64)

x1 = seqs1h_test[:10]
x2 = F.one_hot(seqs2).type(torch.float32).to(output_device)
x2 = torch.flatten(x2, 1)

/scratch/local/47956094/ipykernel_80633/3846775826.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seqs1h_test = torch.flatten(torch.tensor(F.one_hot(seqs), dtype=torch.float32), start_dim=1).to(output_device)
